In [1]:
from pgmpy.readwrite import BIFReader
import numpy as np
import random

ModuleNotFoundError: No module named 'pgmpy'

In [8]:
net = BIFReader("asia.bif")

In [9]:
net.get_edges()

[['asia', 'tub'],
 ['smoke', 'lung'],
 ['smoke', 'bronc'],
 ['lung', 'either'],
 ['tub', 'either'],
 ['either', 'xray'],
 ['bronc', 'dysp'],
 ['either', 'dysp']]

In [14]:
net.get_values()

{'asia': array([[0.01],
        [0.99]]),
 'tub': array([[0.05, 0.01],
        [0.95, 0.99]]),
 'smoke': array([[0.5],
        [0.5]]),
 'lung': array([[0.1 , 0.01],
        [0.9 , 0.99]]),
 'bronc': array([[0.6, 0.3],
        [0.4, 0.7]]),
 'either': array([[1., 1., 1., 0.],
        [0., 0., 0., 1.]]),
 'xray': array([[0.98, 0.05],
        [0.02, 0.95]]),
 'dysp': array([[0.9, 0.8, 0.7, 0.1],
        [0.1, 0.2, 0.3, 0.9]])}

In [33]:
net.get_states()

{'asia': ['yes', 'no'],
 'tub': ['yes', 'no'],
 'smoke': ['yes', 'no'],
 'lung': ['yes', 'no'],
 'bronc': ['yes', 'no'],
 'either': ['yes', 'no'],
 'xray': ['yes', 'no'],
 'dysp': ['yes', 'no']}

In [95]:
net.get_parents()['either']

['lung', 'tub']

In [39]:
def get_children(net, n):

    parents = net.get_parents()
    l = []
    for node in parents:
        if n in parents[node]:
            l.append(node)
    return l

get_children(net, 'lung')

['either']

In [94]:
def gen_sample(net):
    states = net.get_states()
    l = dict()
    for node in states:
        cardinality = len(states[node]) - 1 
        l[node] = random.randint(0, cardinality)
        
    return l
    
#print(net.get_values()['asia']*net.get_values()['either'])
#joint_prob(net, 'tub')
sample = gen_sample(net)
sample

{'asia': 1,
 'tub': 0,
 'smoke': 1,
 'lung': 0,
 'bronc': 0,
 'either': 0,
 'xray': 0,
 'dysp': 0}

In [10]:
net.get_parents()

{'asia': [],
 'tub': ['asia'],
 'smoke': [],
 'lung': ['smoke'],
 'bronc': ['smoke'],
 'either': ['lung', 'tub'],
 'xray': ['either'],
 'dysp': ['bronc', 'either']}

In [107]:
def joint_prob(net, n, sample):
    values = net.get_values()
    
    parents = net.get_parents()[n]
    children = get_children(net,n)
    
    value = values[n]
    print(n, '\n', value, '\n')
    
    if len(parents) > 0:
        
        for i in len(value):
            parents[i]
            
            value[i] = value[i]
        
    
    
    
    
        
        
    #for node in children:
    #    print(node)
    #    print(values[node])

def gibbs_sampling(net, sample, n_iter=1, debug=1):
    for i in range(n_iter):
        
        # for each value in the sample
        for j in sample:
            parents = net.get_parents()[j]
            children = get_children(net, j)
            #if debug: print(parents + children)
            joint_prob(net, j, sample)
            
        
    
gibbs_sampling(net, sample)    

asia 
 [[0.01]
 [0.99]] 

tub 
 [[0.05 0.01]
 [0.95 0.99]] 

smoke 
 [[0.5]
 [0.5]] 

lung 
 [[0.1  0.01]
 [0.9  0.99]] 

bronc 
 [[0.6 0.3]
 [0.4 0.7]] 

either 
 [[1. 1. 1. 0.]
 [0. 0. 0. 1.]] 

xray 
 [[0.98 0.05]
 [0.02 0.95]] 

dysp 
 [[0.9 0.8 0.7 0.1]
 [0.1 0.2 0.3 0.9]] 



In [101]:
net.get_parents()['smoke']

[]

In [46]:
net.get_parents()['either']

['lung', 'tub']

In [55]:
asia_model = net.get_model()

In [56]:
from pgmpy.inference import VariableElimination
asia_infer = VariableElimination(asia_model)

In [121]:
# Computing the probability of bronc given smoke.
q = asia_infer.query(variables=['tub'])
print(q)

Eliminating: asia: 100%|██████████| 7/7 [00:00<00:00, 320.73it/s]

+----------+------------+
| tub      |   phi(tub) |
+==========+============+
| tub(yes) |     0.0104 |
+----------+------------+
| tub(no)  |     0.9896 |
+----------+------------+


In [71]:
from pgmpy.models.BayesianModel import BayesianModel
from pgmpy.sampling import GibbsSampling
gibbs = GibbsSampling(asia_model)
gibbs.generate_sample(size=10, return_type='dataframe')

KeyError: 0

In [63]:
asia_model.check_model()

True

In [120]:
print(asia_model.get_cpds()[3])
print(asia_model.get_cpds()[6])

+-------------+-----------+-----------+----------+----------+
| lung        | lung(yes) | lung(yes) | lung(no) | lung(no) |
+-------------+-----------+-----------+----------+----------+
| tub         | tub(yes)  | tub(no)   | tub(yes) | tub(no)  |
+-------------+-----------+-----------+----------+----------+
| either(yes) | 1.0       | 1.0       | 1.0      | 0.0      |
+-------------+-----------+-----------+----------+----------+
| either(no)  | 0.0       | 0.0       | 0.0      | 1.0      |
+-------------+-----------+-----------+----------+----------+
+----------+-----------+----------+
| asia     | asia(yes) | asia(no) |
+----------+-----------+----------+
| tub(yes) | 0.05      | 0.01     |
+----------+-----------+----------+
| tub(no)  | 0.95      | 0.99     |
+----------+-----------+----------+


In [110]:
asia_model.get_markov_blanket('lung')

['either', 'tub', 'smoke']

In [114]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
G = BayesianModel([('x', 'y'), ('z', 'y'), ('y', 'w'), ('y', 'v'), ('u', 'w'),
                       ('s', 'v'), ('w', 't'), ('w', 'm'), ('v', 'n'), ('v', 'q'), ('z','a')])
G.get_markov_blanket('y')

['x', 'z', 'w', 'u', 's', 'v']

In [115]:
asia_model.get_cpds()

[<TabularCPD representing P(asia:2) at 0x7f773a5adb10>,
 <TabularCPD representing P(bronc:2 | smoke:2) at 0x7f773a5adcd0>,
 <TabularCPD representing P(dysp:2 | bronc:2, either:2) at 0x7f773a5add90>,
 <TabularCPD representing P(either:2 | lung:2, tub:2) at 0x7f773a5ad090>,
 <TabularCPD representing P(lung:2 | smoke:2) at 0x7f773a5add50>,
 <TabularCPD representing P(smoke:2) at 0x7f773a602250>,
 <TabularCPD representing P(tub:2 | asia:2) at 0x7f773a547d50>,
 <TabularCPD representing P(xray:2 | either:2) at 0x7f773a547f50>]

In [1]:
import read_bayesnet
from read_bayesnet import BayesianNetwork, Variable

In [3]:
bn = BayesianNetwork(file='asia.bif')  # example usage for the supplied earthquake.bif file
for v in bn.variables:
    print('node : ',v.name,'\nparents : ',v.parents,'\nmarkov blanket : ',v.markov_blanket,'\nprobs : ',v.probabilities,'\n')

node :  asia 
parents :  [] 
markov blanket :  ['tub'] 
probs :  {'yes': 0.01, 'no': 0.99} 

node :  tub 
parents :  ['asia'] 
markov blanket :  ['asia', 'lung', 'either'] 
probs :  {('yes',): {'yes': 0.05, 'no': 0.95}, ('no',): {'yes': 0.01, 'no': 0.99}} 

node :  smoke 
parents :  [] 
markov blanket :  ['lung', 'bronc'] 
probs :  {'yes': 0.5, 'no': 0.5} 

node :  lung 
parents :  ['smoke'] 
markov blanket :  ['tub', 'smoke', 'either'] 
probs :  {('yes',): {'yes': 0.1, 'no': 0.9}, ('no',): {'yes': 0.01, 'no': 0.99}} 

node :  bronc 
parents :  ['smoke'] 
markov blanket :  ['smoke', 'either', 'dysp'] 
probs :  {('yes',): {'yes': 0.6, 'no': 0.4}, ('no',): {'yes': 0.3, 'no': 0.7}} 

node :  either 
parents :  ['lung', 'tub'] 
markov blanket :  ['tub', 'dysp', 'bronc', 'lung', 'xray'] 
probs :  {('yes', 'yes'): {'yes': 1.0, 'no': 0.0}, ('no', 'yes'): {'yes': 1.0, 'no': 0.0}, ('yes', 'no'): {'yes': 1.0, 'no': 0.0}, ('no', 'no'): {'yes': 0.0, 'no': 1.0}} 

node :  xray 
parents :  ['either'